In [6]:
from matplotlib import pyplot as plt
import numpy as np

#TO-DO: Get accurate estimations for current values

# Request deployment duration
duration = input("Deployment Duration:")

# Define processing dictionary
processing_options = {
    "Raw Recording": 5,   # mA
    "Inference": 18,
    "Filtering + Compression": 20
}

# Prompt processing method
print("\nPlease choose processing method:")
print("Raw Recording")
print("Inference")
print("Filtering + Compression")
choice = input("Enter your choice: ")

# Add to current consumption
current_consumption += processing_options[choice]
print(current_consumption)

# Define output dictionary
output_options = {
    "Save to SD": 2,       # mA
    "Transmit via LoRa": 25
}

# Prompt output method
print("\nPlease choose output method:")
print("Save to SD")
print("Transmit via LoRa")
choice = input("Enter your choice: ")

# Add to current consumption
current_consumption += output_options[choice]
print(current_consumption)

# Define model dictionary
model_options = {
    "Burrowing Owl Model": 15,  # mA
    "Panda Bear Model": 10,
    "Polar Bear Model": 25
}

# Prompt output method
print("\nPlease choose model option:")
print("Panda Bear Model")
print("Polar Bear Model")
choice = input("Enter your choice: ")

# Add to current consumption
current_consumption += output_options[choice]
print(current_consumption)


# Output desired plot
#fig, ax = plt.subplots()

Deployment Duration: 5



Please choose processing method:
Raw Recording
Inference
Filtering + Compression


Enter your choice:  Raw Recording


10

Please choose output method:
Save to SD
Transmit via LoRa


Enter your choice:  fd


KeyError: 'fd'